# Analysing data on hospital buildings (ERIC) - backlogs

The Estates Returns Information Collection (ERIC) - data on NHS buildings including hospitals - is [published every October](https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection).

This notebook details the code needed to:

* Fetch the data from 5 years of spreadsheets
* Drill down to the columns on repair backlogs
* Combine the data
* Clean
* Export for analysis

The [page publishing the data](https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/estates-return-information-collection-2016-17) notes:

> Note: in 2019 we were advised of an error in Devonshire Partnership NHS Trust's submitted Oil Consumption figures. The correct figure for the aggregate site consumption is 40,798.8 kWh, rather than the reported 3,855 kWh.

> Note: 7th September 2021: When the revalidated data was released, only the revised headline figures, report (containing trust, site and PFI level data) and data quality statement were made available (figures in the underlying data .csv files were not updated, although revised trust, site and PFI revised figures were available in the data tables). We apologise for any confusion caused and have now published revised .csv files to accompany the release products. These are clearly labelled below.

For this analysis we are using the data marked "revalidation".


## Import the libraries

First we need to import the libraries needed.

In [ ]:
#import pandas for dealing with data
import pandas as pd
#we will need the math library too for detecting nan values
import math
#requests for fetching URLs
import requests
#beautiful soup for drilling into them
from bs4 import BeautifulSoup

In [ ]:
#import re for regex
import re

## Change default formatting

Let's [suppress scientific notation for larger numbers by specifying some default formatting](https://saturncloud.io/blog/how-to-suppress-scientific-notation-in-pandas)

In [ ]:
# change default formatting to not use scientific notation
pd.options.display.float_format = '{:,.3f}'.format

## Create functions to filter...

In a [previous notebook](https://colab.research.google.com/drive/1B7hT6PDdO-XZigGiI3n_iKNRRRN293cE?usp=sharing) we explored the data and codified that in some functions. Let's recreate those here.

The first one takes a large ERIC spreadsheet and filters it to just the key columns and those on backlogs. It also filters out non-numbers.

In [ ]:
#define a function, it takes one argument - the url of the CSV
def backlogdataonly(csvurl):
  #read in the CSV
  sitedata = pd.read_csv(csvurl, encoding = "ISO-8859-1")
  #store the first 10 column names
  #until 22/23 we wanted the first 9 but in that year the Site Type column was pushed along to column J
  #and they added Local Authority in column I
  keykeys = list(sitedata.keys()[0:10])
  print(keykeys)
  #loop through the keys and extract the ones with backlog in them
  backlog_keys = [key for key in sitedata.keys() if 'backlog' in key.lower()]
  #add those keys to the ones we've already stored
  bothkeys = keykeys[:10]+backlog_keys
  print(bothkeys)
  #use those to extract a subset
  backlogdf = sitedata[bothkeys]
  #reshape from wide to long
  longversion = pd.melt(backlogdf, id_vars=list(sitedata.keys()[0:10]),var_name='measure', value_name='values')
  #print(longversion)
  #filter to the rows where the condition is True
  backlog_filtered = longversion.drop(longversion[[type(i) == float for i in longversion["values"]]].index)
  #.index converts that list of T/F to a list of indices
  backlog_filtered = backlog_filtered.drop(backlog_filtered[backlog_filtered['values'] == 'Not Applicable'].index)
  #remove the extra row of headers too - this time inplace
  backlog_filtered = backlog_filtered.drop(backlog_filtered[backlog_filtered['values'] == 'Cost to eradicate high risk backlog (£)'].index)
  #rename columns where name has extra chars
  if 'Trust Code' in backlog_filtered.keys()[0]:
    print('HEY', backlog_filtered.keys()[0])
    replacename = backlog_filtered.keys()[0]
    print('HEYHEY', replacename)
    backlog_filtered = backlog_filtered.rename(columns={replacename: 'Trust Code'})
  if 'New Commissioning Region' in backlog_filtered.keys()[3]:
    print('HEY', backlog_filtered.keys()[3])
    replacename = backlog_filtered.keys()[3]
    print('HEYHEY', replacename)
    backlog_filtered = backlog_filtered.rename(columns={replacename: 'Commissioning Region'})
  #print(backlogdf.keys())
  #return the resulting dataframe to whatever called the function
  return(backlog_filtered)

### ...And clean strings to numbers

The second function converts the values of a column from strings to numbers.

In [ ]:
def cleannumbers(column):
  #create a new list
  column_as_ints = []
  #loop through the strings
  for i in column:
    #print(i)
    #if it's a string, which they all should be now
    if type(i) == str:
      #replace the comma, otherwise it won't convert to an integer
      newfigure = int(i.replace(',',''))
      #add to the list
      column_as_ints.append(int(newfigure))
    else:
      print('HUH', type(i))
      print(math.isnan(i))
  return(column_as_ints)

## Apply the functions to 5 years of data

Now let's store the URLs of each dataset. We start with the homepage where they can all be accessed.

In [ ]:
#store the ERIC homepage
ericurl = "https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection"
#store the base URL which we will need for relative URLs
baseurl = "https://digital.nhs.uk"

### Create a function to scrape the data CSV

This function will find the 'site data' CSV link on each page.

In [ ]:
#define a function that takes a URL and returns the site data CSV link on that page
def fetchcsv_for_sites(url):
  # Send a GET request to the link URL
  link_response = requests.get(url)
  #parse into soup
  soup = BeautifulSoup(link_response.content, 'html.parser')
  # Find all links
  divboxlink = soup.find_all('a')
  #create an empty list
  matches = []
  #loop through each one
  for i in divboxlink:
    #look for the one about Site data
    if "Site" in i.get('href'):
      #show that URL
      #print(i.get('href'))
      matches.append(i.get('href'))
  #if the list has something in it
  if len(matches) >0:
    #return that URL
    return(matches[0])
  #otherwise
  else:
    #return a string we can pick up the other side
    return('NO LINK')

In [ ]:
#create an empty list to store the URLs
csvurls = []

#some of this code generated by ChatGPT in response to the prompt:
#"write some python code which identifies the first link inside a <h3> tag at
#https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection and fetches that"
#fetch that page
response = requests.get(ericurl)
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')
# Find all the <h3> tags
h3_tag = soup.find_all('h3')
#loop through the last 5 years
for i in h3_tag[:5]:
  #find the first <a> and get the href= attribute
  yearpageurl = baseurl+i.find('a').get('href')
  print(yearpageurl)
  #run the function defined above to fetch the CSV link from that page
  sitedatacsvurl = fetchcsv_for_sites(yearpageurl)
  print(sitedatacsvurl)
  #add it to the list unless it's a 'NO LINK'
  if sitedatacsvurl != 'NO LINK':
    csvurls.append(sitedatacsvurl)


print(csvurls)

https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/england-2022-23
https://files.digital.nhs.uk/41/5787C9/ERIC%20-%202022_23%20-%20Site%20data.csv
https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/england-2021-22
https://files.digital.nhs.uk/EE/7E330D/ERIC%20-%20202122%20-%20Site%20Data%20v3.csv
https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/england-2020-21
https://files.digital.nhs.uk/0F/46F719/ERIC%20-%20202021%20-%20Site%20data%20v2.csv
https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/england-2019-20
https://files.digital.nhs.uk/11/BC1043/ERIC%20-%20201920%20-%20SiteData%20-%20v2.csv
https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/england-2018-19
https://files.digital.nhs.uk/63/ADBFFF/ERIC%20-

### Loop through the CSV urls

In [ ]:
csvurls

['https://files.digital.nhs.uk/41/5787C9/ERIC%20-%202022_23%20-%20Site%20data.csv',
 'https://files.digital.nhs.uk/EE/7E330D/ERIC%20-%20202122%20-%20Site%20Data%20v3.csv',
 'https://files.digital.nhs.uk/0F/46F719/ERIC%20-%20202021%20-%20Site%20data%20v2.csv',
 'https://files.digital.nhs.uk/11/BC1043/ERIC%20-%20201920%20-%20SiteData%20-%20v2.csv',
 'https://files.digital.nhs.uk/63/ADBFFF/ERIC%20-%20201819%20-%20SiteData%20v4.csv']

In [ ]:
#test the function on the 22/23 data because it's returning Site Type as NaN
thisyrdf = backlogdataonly('https://files.digital.nhs.uk/41/5787C9/ERIC%20-%202022_23%20-%20Site%20data.csv')

['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board', 'Local Authority', 'Site Type']
['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board', 'Local Authority', 'Site Type', 'Cost to eradicate high risk backlog (£)', 'Cost to eradicate significant risk backlog (£)', 'Cost to eradicate moderate risk backlog (£)', 'Cost to eradicate low risk backlog (£)', 'Percentage of GIA surveyed using risk adjusted backlog guidance (Select)', 'Methodology used to review costs to eradicate backlog (Select)', 'Methodology used to review costs to eradicate backlog - Reason (Notes)', 'Investment to reduce backlog maintenance - Critical Infrastructure Risk (£)', 'Investment to reduce backlog maintenance - non Critical Infrastructure Risk (£)']
HEY Trust Code
HEYHEY Trust Code


In [ ]:
#create an empty dataframe
last5yrs = pd.DataFrame()

#loop through the URLs
for i in csvurls:
  print(i)
  thisyrdf = backlogdataonly(i)
  #extract the year from the URL
  thisyrdf['year_range'] = i.split('ERIC')[1].split('-')[1].split('-')[0].replace('%20','').replace('_','')
  last5yrs = last5yrs.append(thisyrdf, ignore_index = True)



https://files.digital.nhs.uk/41/5787C9/ERIC%20-%202022_23%20-%20Site%20data.csv
['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board', 'Local Authority', 'Site Type']
['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board', 'Local Authority', 'Site Type', 'Cost to eradicate high risk backlog (£)', 'Cost to eradicate significant risk backlog (£)', 'Cost to eradicate moderate risk backlog (£)', 'Cost to eradicate low risk backlog (£)', 'Percentage of GIA surveyed using risk adjusted backlog guidance (Select)', 'Methodology used to review costs to eradicate backlog (Select)', 'Methodology used to review costs to eradicate backlog - Reason (Notes)', 'Investment to reduce backlog maintenance - Critical Infrastructure Risk (£)', 'Investment to reduce backlog maintenance - non Critical Infrastructure Risk (£)']
HEY Trust Code
HEYHEY Trust Code
h

<ipython-input-11-20f8a91b48e2>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last5yrs = last5yrs.append(thisyrdf, ignore_index = True)


['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Status', 'Site Code', 'Site Name', 'Post Code', 'Site Type', 'Tenure']
['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Status', 'Site Code', 'Site Name', 'Post Code', 'Site Type', 'Tenure', 'Cost to eradicate high risk backlog (£)', 'Cost to eradicate significant risk backlog (£)', 'Cost to eradicate moderate risk backlog (£)', 'Cost to eradicate low risk backlog (£)', 'Percentage of GIA surveyed using risk adjusted backlog guidance (Select)', 'Methodology used to review costs to eradicate backlog (Select)', 'Methodology used to review costs to eradicate backlog - Reason (Notes)', 'Investment to reduce backlog maintenance - Critical Infrastructure Risk (£)', 'Investment to reduce backlog maintenance - non Critical Infrastructure Risk (£)']
HEY Trust Code
HEYHEY Trust Code
https://files.digital.nhs.uk/0F/46F719/ERIC%20-%20202021%20-%20Site%20data%20v2.csv


<ipython-input-11-20f8a91b48e2>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last5yrs = last5yrs.append(thisyrdf, ignore_index = True)


['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Site Type', 'Tenure', 'Leased', 'Post Code']
['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Site Type', 'Tenure', 'Leased', 'Post Code', 'Cost to eradicate high risk backlog (£)', 'Cost to eradicate significant risk backlog (£)', 'Cost to eradicate moderate risk backlog (£)', 'Cost to eradicate low risk backlog (£)', 'Percentage of GIA that has had a risk adjusted backlog review (Select)', 'Methodology used to review costs to eradicate backlog (Select)', 'Methodology used to review costs to eradicate backlog - Reason (Notes)']
HEY Trust Code
HEYHEY Trust Code
https://files.digital.nhs.uk/11/BC1043/ERIC%20-%20201920%20-%20SiteData%20-%20v2.csv


<ipython-input-11-20f8a91b48e2>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last5yrs = last5yrs.append(thisyrdf, ignore_index = True)


['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Site Type', 'Tenure', 'Leased', 'Post Code']
['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Site Type', 'Tenure', 'Leased', 'Post Code', 'Cost to eradicate high risk backlog (£)', 'Cost to eradicate significant risk backlog (£)', 'Cost to eradicate moderate risk backlog (£)', 'Cost to eradicate low risk backlog (£)']
HEY Trust Code
HEYHEY Trust Code
https://files.digital.nhs.uk/63/ADBFFF/ERIC%20-%20201819%20-%20SiteData%20v4.csv


<ipython-input-11-20f8a91b48e2>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last5yrs = last5yrs.append(thisyrdf, ignore_index = True)


['Trust Code', 'Trust Name', 'Old Commissioning Region', 'New Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Site Type', 'Tenure', 'Leasehold Type']
['Trust Code', 'Trust Name', 'Old Commissioning Region', 'New Commissioning Region', 'Trust Type', 'Site Code', 'Site Name', 'Site Type', 'Tenure', 'Leasehold Type', 'Cost to eradicate high risk backlog (£)', 'Cost to eradicate significant risk backlog (£)', 'Cost to eradicate moderate risk backlog (£)', 'Cost to eradicate low risk backlog (£)']
HEY Trust Code
HEYHEY Trust Code
HEY New Commissioning Region
HEYHEY New Commissioning Region


<ipython-input-11-20f8a91b48e2>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last5yrs = last5yrs.append(thisyrdf, ignore_index = True)


In [ ]:
last5yrs

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,measure,values,year_range,Status,Tenure,Leased,Old Commissioning Region,Leasehold Type
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN
3,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A01,ISLAND SITE,M13 9WL,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,General acute hospital,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN
4,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A06,UNIVERSITY DENTAL HOSPITAL OF MANCHESTER,M15 6FH,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46048,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ07,EDWARD STREET HOSPITAL,NaN,NaN,NaN,4. Mental Health (including Specialist services),Cost to eradicate low risk backlog (£),"219,626",201819,NaN,1. Freehold,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN
46049,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ11,HEATH LANE HOSPITAL,NaN,NaN,NaN,3. Mixed service hospital,Cost to eradicate low risk backlog (£),"23,875",201819,NaN,1. Freehold,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN
46050,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ20,HALLAM STREET HOSPITAL,NaN,NaN,NaN,3. Mixed service hospital,Cost to eradicate low risk backlog (£),"26,262",201819,NaN,2. Whole site - Private Finance Initiative (PFI),NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN
46051,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ52,PENN HOSPITAL,NaN,NaN,NaN,4. Mental Health (including Specialist services),Cost to eradicate low risk backlog (£),"278,156",201819,NaN,1. Freehold,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN


### Clean the backlog measure text

The column headings in the original data are stored as values in this long dataframe, but there are inconsistencies in the way the columns were titled: some are 'Cost to eradicate high risk backlog (√Ç¬£)' and some are 'Cost to eradicate high risk backlog (¬£)' - below those inconsistencies are tackled and the data is cleaned.

In [ ]:
#create a new empty list
newlist = []

#loop through the measure column
for i in last5yrs['measure']:
  #print(i)
  #clean up the entries where there's an inconsistently garbled £ sign
  if 'Cost to eradicate high risk backlog' in i:
    #and replace those entries with a non-garbled consistent version
    i = 'Cost to eradicate high risk backlog (GBP)'
  elif 'Cost to eradicate low risk backlog' in i:
    i = 'Cost to eradicate low risk backlog (GBP)'
  elif 'Cost to eradicate moderate risk backlog' in i:
    i = 'Cost to eradicate moderate risk backlog (GBP)'
  elif 'Cost to eradicate significant risk backlog' in i:
    i = 'Cost to eradicate significant risk backlog (GBP)'
  #add to the previously empty list, which can then be used as a new column
  newlist.append(i)

#create new column from that list
last5yrs['measureclean'] = newlist

### Remove other measures

We only want the backlog data but the process has captured some other measures as well. We can use `.unique()` to show what measures there are.

In [ ]:
last5yrs['measureclean'].unique()

array(['Cost to eradicate high risk backlog (GBP)',
       'Cost to eradicate significant risk backlog (GBP)',
       'Cost to eradicate moderate risk backlog (GBP)',
       'Cost to eradicate low risk backlog (GBP)',
       'Percentage of GIA surveyed using risk adjusted backlog guidance (Select)',
       'Methodology used to review costs to eradicate backlog (Select)',
       'Methodology used to review costs to eradicate backlog - Reason (Notes)',
       'Investment to reduce backlog maintenance - Critical Infrastructure Risk (£)',
       'Investment to reduce backlog maintenance - non Critical Infrastructure Risk (£)',
       'Percentage of GIA that has had a risk adjusted backlog review (Select)'],
      dtype=object)

Now we use `.drop()` to drop rows that do not have the key phrase we want.

In [ ]:
#filter to the rows where the condition is True
last5yrs_filtered = last5yrs.drop(last5yrs[['Cost to eradicate' not in i for i in last5yrs["measureclean"]]].index)
last5yrs_filtered['measureclean'].unique()


array(['Cost to eradicate high risk backlog (GBP)',
       'Cost to eradicate significant risk backlog (GBP)',
       'Cost to eradicate moderate risk backlog (GBP)',
       'Cost to eradicate low risk backlog (GBP)'], dtype=object)

In [ ]:
#overwrite the original dataframe
last5yrs = last5yrs_filtered


In [ ]:
last5yrs

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,measure,values,year_range,Status,Tenure,Leased,Old Commissioning Region,Leasehold Type,measureclean
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP)
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP)
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP)
3,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A01,ISLAND SITE,M13 9WL,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,General acute hospital,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP)
4,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A06,UNIVERSITY DENTAL HOSPITAL OF MANCHESTER,M15 6FH,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,Cost to eradicate high risk backlog (£),0,202223,NaN,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46048,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ07,EDWARD STREET HOSPITAL,NaN,NaN,NaN,4. Mental Health (including Specialist services),Cost to eradicate low risk backlog (£),"219,626",201819,NaN,1. Freehold,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP)
46049,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ11,HEATH LANE HOSPITAL,NaN,NaN,NaN,3. Mixed service hospital,Cost to eradicate low risk backlog (£),"23,875",201819,NaN,1. Freehold,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP)
46050,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ20,HALLAM STREET HOSPITAL,NaN,NaN,NaN,3. Mixed service hospital,Cost to eradicate low risk backlog (£),"26,262",201819,NaN,2. Whole site - Private Finance Initiative (PFI),NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP)
46051,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ52,PENN HOSPITAL,NaN,NaN,NaN,4. Mental Health (including Specialist services),Cost to eradicate low risk backlog (£),"278,156",201819,NaN,1. Freehold,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP)


### Clean the values from strings to numbers

We created a function to clean the values column, but haven't yet used it because before now it would have returned an error when it tried to clean (string) values for other measures.

With those measures removed, we can now apply it.

In [ ]:
#create new column based on applying the custom function to another column
last5yrs['valuesclean'] = cleannumbers(last5yrs['values'])

In [ ]:
last5yrs.to_csv("aftercleaningvalues.csv")

### Test a pivot table

We can test it by generating a pivot table on the cost of high risk backlogs by commissioning region, and check if the values match what we get by doing the same in Excel (they do).

In [ ]:
#check the keys we can use
last5yrs.keys()

Index(['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type',
       'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board',
       'Local Authority', 'Site Type', 'measure', 'values', 'year_range',
       'Status', 'Tenure', 'Leased', 'Old Commissioning Region',
       'Leasehold Type', 'measureclean', 'valuesclean'],
      dtype='object')

In [ ]:
last5yrs[(last5yrs.measureclean == 'Cost to eradicate high risk backlog (GBP)')].pivot_table(
    index="Commissioning Region",
    values="valuesclean",
    aggfunc="sum",
    columns = "year_range")


year_range,201819,201920,202021,202122,202223
Commissioning Region,,,,,
EAST OF ENGLAND COMMISSIONING REGION,82851933,206879392,231756919,250338083,292944001
LONDON COMMISSIONING REGION,594844291,736486080,696406511,807397991,935014807
MIDLANDS COMMISSIONING REGION,82879960,116401230,148178288,144206654,158961697
NORTH EAST AND YORKSHIRE COMMISSIONING REGION,80787100,143125843,166150226,266713091,520291401
NORTH WEST COMMISSIONING REGION,90411230,127389026,65653067,130548615,107296917
SOUTH EAST COMMISSIONING REGION,135148302,128128104,216080176,150003974,287031629
SOUTH WEST COMMISSIONING REGION,28044600,46908048,52721856,55369804,61483522


### Clean the site types

Until 2021/22 the 'Site type' column included ordinal prefixes, e.g. `1. General acute hospital`. But the most recent data is not numbered.

We can clean the data so that it's consistent and doesn't need cleaning later.

In [ ]:
#pivot on the Site Type field
last5yrs[(last5yrs.measureclean == 'Cost to eradicate high risk backlog (GBP)')].pivot_table(
    index="Site Type",
    values="valuesclean",
    aggfunc="sum",
    columns = "year_range")


year_range,201819,201920,202021,202122,202223
Site Type,,,,,
1. General acute hospital,"1,011,977,199.000","1,344,086,371.000","1,337,342,227.000",NaN,NaN
2. Specialist hospital (acute only),"26,909,470.000","57,614,930.000","111,438,995.000",NaN,NaN
3. Mixed service hospital,"16,015,923.000","9,042,227.000","17,845,479.000",NaN,NaN
4. Mental Health (including Specialist services),"12,326,768.000","15,572,501.000","21,467,329.000",NaN,NaN
5. Learning Disabilities,"147,300.000","132,400.000","88,999.000",NaN,NaN
6. Mental Health and Learning Disabilities,"405,360.000","465,697.000","175,010.000",NaN,NaN
7. Community hospital (with inpatient beds),"6,814,885.000","35,980,951.000","36,589,307.000",NaN,NaN
8. Other inpatient,"173,000.000","578,080.000","422,142.000",NaN,NaN
Ambulance services,NaN,"1,221,694.000","430,346.000",NaN,NaN


In [ ]:
#testing line
#re.sub('[0-9]\. ','','1. Learning Disabilities')
sitetypeclean = [re.sub('[0-9]\. ','',i) for i in last5yrs['Site Type']]
sitetypeclean[:10]

['Non inpatient',
 'Non inpatient',
 'Non inpatient',
 'General acute hospital',
 'Non inpatient',
 'General acute hospital',
 'Non inpatient',
 'General acute hospital',
 'Mixed service hospital',
 'Other inpatient']

### Add back into dataframe and pivot



In [ ]:
#create a new column from that list
last5yrs['sitetypeclean'] = sitetypeclean

In [ ]:
#pivot on that field
last5yrs[(last5yrs.measureclean == 'Cost to eradicate high risk backlog (GBP)')].pivot_table(
    index="sitetypeclean",
    values="valuesclean",
    aggfunc="sum",
    columns = "year_range")


year_range,201819,201920,202021,202122,202223
sitetypeclean,,,,,
Ambulance services,NaN,"1,221,694.000","430,346.000",NaN,NaN
Community hospital (with inpatient beds),"6,814,885.000","35,980,951.000","36,589,307.000","64,246,402.000","25,166,629.000"
General acute hospital,"1,011,977,199.000","1,344,086,371.000","1,337,342,227.000","1,502,525,057.000","2,078,970,702.000"
Learning Disabilities,"147,300.000","132,400.000","88,999.000","904,407.000","201,000.000"
Mental Health (including Specialist services),"12,326,768.000","15,572,501.000","21,467,329.000","34,136,072.000","21,101,866.000"
Mental Health and Learning Disabilities,"405,360.000","465,697.000","175,010.000","13,245,547.000","7,481,174.000"
Mixed service hospital,"16,015,923.000","9,042,227.000","17,845,479.000","55,309,005.000","48,189,877.000"
Non inpatient,NaN,NaN,NaN,"37,297,440.000","50,419,403.000"
Other Reportable Site,"20,197,511.000","40,622,872.000","51,147,209.000","11,137,100.000","31,604,931.000"


In [ ]:
last5yrs.to_csv("aftercleaningsites.csv")

### Add a 'year ending' column

For the visualisation we need a year ending column rather than '202122' so let's create that too.

In [ ]:
#grab the last two digits of every string in year_range
#add '20' to the front of those, and store in a list
yearending = ['20'+i[-2:] for i in last5yrs['year_range']]

#add to dataframe
last5yrs['yearending'] = yearending

## Export as a CSV

In [ ]:
#create a CSV from the dataframe
last5yrs.to_csv('last5yrs.csv')
#import a library for downloading files
from google.colab import files
#download the file
files.download('last5yrs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Adjust for inflation (GDP)

As these are financial figures covering multiple years, we should adjust them for inflation: £1 five years ago is not the same as £1 last year.

To do that we need to:

1. Import values that can be used to adjust (multiply) a value for a list of given periods (financial years)
2. Merge those values with our existing data, matching on the period the values relate to
3. Perform a calculation converting the historical values to a value at a particular point in time (the most recent period)
4. Store the results in a new column



### Import and clean up inflation adjustment data

A 'GDP deflator' table used for a previous story has been put in a dedicated Google Sheet and published as a CSV. We import that to use it in calculations.

In [ ]:
#store the URL of the published spreadsheet
gdpconverterurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQg4KXqQ0E2uO3hRRCDFOvhNQEM1Dc4y7P5OF-9ETiQocP4qygYzzl-i4E4fBkPdhytN-CqvBLqeWpf/pub?gid=0&single=true&output=csv"
#import that into a dataframe - just the last two columns and 11 rows
gdpdeflatordf = pd.read_csv(gdpconverterurl,
                            usecols=[3,4],
                            nrows = 11)
#show it
gdpdeflatordf

,Financial Year,GDP Deflator.1
0,2012-13,78.901
1,2013-14,80.542
2,2014-15,81.428
3,2015-16,82.077
4,2016-17,83.762
5,2017-18,85.159
6,2018-19,86.680
7,2019-20,88.934
8,2020-21,94.507
9,2021-22,93.852


### Convert column to match ours

We then need to make it consistent with our existing data.

Our existing data uses a column called `year_range` with values like `202122` so we need this data to match.

In [ ]:
#create a new column to match on
gdpdeflatordf['year_range'] = [str(i).replace('-','') for i in gdpdeflatordf['Financial Year']]
#we don't need to convert to integer
#gdpdeflatordf['year_range'] = [int(i) for i in gdpdeflatordf['year_range']]
#show
gdpdeflatordf

,Financial Year,GDP Deflator.1,year_range
0,2012-13,78.901,201213
1,2013-14,80.542,201314
2,2014-15,81.428,201415
3,2015-16,82.077,201516
4,2016-17,83.762,201617
5,2017-18,85.159,201718
6,2018-19,86.680,201819
7,2019-20,88.934,201920
8,2020-21,94.507,202021
9,2021-22,93.852,202122


We can also test the data types match. `O` means 'object'.

In [ ]:
last5yrs['year_range'].dtype

dtype('O')

In [ ]:
gdpdeflatordf['year_range'].dtype

dtype('O')

### Merge the two dataframes

Now we can merge them.

In [ ]:
#merge the two dataframes on the year_range column and store in a new df
last5yrs_wdeflator = pd.merge(left = last5yrs,
                              right = gdpdeflatordf,
                              on = 'year_range')

last5yrs_wdeflator.head()

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,...,Tenure,Leased,Old Commissioning Region,Leasehold Type,measureclean,valuesclean,sitetypeclean,yearending,Financial Year,GDP Deflator.1
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,...,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000
3,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A01,ISLAND SITE,M13 9WL,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,General acute hospital,...,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,General acute hospital,2023,2022-23,100.000
4,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A06,UNIVERSITY DENTAL HOSPITAL OF MANCHESTER,M15 6FH,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000


### Use the deflator to create a new column

Now the deflator and the financial values are in the same dataframe, we can use them to adjust the values for inflation.

In [ ]:
#create a new column based on multiplying one column by 1 divided by another
last5yrs_wdeflator['inflation_adjustedGDP'] = last5yrs_wdeflator['valuesclean']*(1/last5yrs_wdeflator['GDP Deflator.1'])*100

In [ ]:
#we can test some of the calculations in the last 5 rows
last5yrs_wdeflator

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,...,Leased,Old Commissioning Region,Leasehold Type,measureclean,valuesclean,sitetypeclean,yearending,Financial Year,GDP Deflator.1,inflation_adjustedGDP
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,...,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000
3,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A01,ISLAND SITE,M13 9WL,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,General acute hospital,...,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,General acute hospital,2023,2022-23,100.000,0.000
4,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A06,UNIVERSITY DENTAL HOSPITAL OF MANCHESTER,M15 6FH,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28959,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ07,EDWARD STREET HOSPITAL,NaN,NaN,NaN,4. Mental Health (including Specialist services),...,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP),219626,Mental Health (including Specialist services),2019,2018-19,86.680,"253,375.635"
28960,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ11,HEATH LANE HOSPITAL,NaN,NaN,NaN,3. Mixed service hospital,...,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP),23875,Mixed service hospital,2019,2018-19,86.680,"27,543.839"
28961,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ20,HALLAM STREET HOSPITAL,NaN,NaN,NaN,3. Mixed service hospital,...,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP),26262,Mixed service hospital,2019,2018-19,86.680,"30,297.647"
28962,TAJ,BLACK COUNTRY PARTNERSHIP NHS FOUNDATION TRUST,MIDLANDS COMMISSIONING REGION,CARE TRUST,TAJ52,PENN HOSPITAL,NaN,NaN,NaN,4. Mental Health (including Specialist services),...,NaN,MIDLANDS AND EAST OF ENGLAND COMMISSIONING REGION,NaN,Cost to eradicate low risk backlog (GBP),278156,Mental Health (including Specialist services),2019,2018-19,86.680,"320,899.862"


In [ ]:
last5yrs_wdeflator.to_csv("afterGDP.csv")

## Adjust for inflation (CPI)

The measure used above is just one. Another would be to use CPI.



In [ ]:
#store the URL of the published spreadsheet
cpiadjusturl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ0AekL8eOE5B4MlUxFp7h4sMouACy4196DMptykMotNAMdgwi75x03ksB9gTLe8bVeD1M2mIpK06m5/pub?gid=583682061&single=true&output=csv"
#import that into a dataframe - just the last two columns and 11 rows
cpiadjustdf = pd.read_csv(cpiadjusturl)
#show it
cpiadjustdf

,year_range,AVERAGE of CPI ALL ITEMS
0,198687,48.554
1,198788,50.219
2,198889,52.950
3,198990,56.800
4,199091,61.064
5,199192,62.968
6,199293,64.578
7,199394,65.866
8,199495,67.624
9,199596,69.146


### Merge the data

We get an error if we try to merge the data as it is, because the year_range field is an integer type in the CPI data.

We just need to change that.

In [ ]:
#check data type
print(cpiadjustdf['year_range'].dtype)
#convert to str
cpiadjustdf['year_range'] = [str(i) for i in cpiadjustdf['year_range']]
#check data type
print(cpiadjustdf['year_range'].dtype)

int64
object


In [ ]:
#merge the two dataframes on the year_range column and store in a new df
last5yrs_wdeflator = pd.merge(left = last5yrs_wdeflator,
                              right = cpiadjustdf,
                              on = 'year_range')

last5yrs_wdeflator.head()

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,...,Old Commissioning Region,Leasehold Type,measureclean,valuesclean,sitetypeclean,yearending,Financial Year,GDP Deflator.1,inflation_adjustedGDP,AVERAGE of CPI ALL ITEMS
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,...,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092
3,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A01,ISLAND SITE,M13 9WL,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,General acute hospital,...,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,General acute hospital,2023,2022-23,100.000,0.000,131.092
4,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A06,UNIVERSITY DENTAL HOSPITAL OF MANCHESTER,M15 6FH,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,NaN,NaN,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092


### Use the CPI figures to create a new column

Whereas we just multiplied by 100 with the GDP (because it was indexed to the current year), the CPI data needs to be multiplied by the current year's value (it is indexed to 2015).

This is in the last row.

In [ ]:
list(cpiadjustdf['AVERAGE of CPI ALL ITEMS'])[-1]

131.0922

In [ ]:
#store the value for the current year
currentCPI = list(cpiadjustdf['AVERAGE of CPI ALL ITEMS'])[-1]
#create a new column based on multiplying one column by 1 divided by another
#multiplied by the current CPI
last5yrs_wdeflator['inflation_adjustedCPI'] = last5yrs_wdeflator['valuesclean']*(1/last5yrs_wdeflator['AVERAGE of CPI ALL ITEMS'])*currentCPI

### Export the inflation-adjusted data as a CSV

In [ ]:
#export as a CSV
last5yrs_wdeflator.to_csv('last5yrs_wdeflator.csv')
#download the file
files.download('last5yrs_wdeflator.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Adjust for inflation (construction OPI)

We can adjust for a more specific category of inflation: The ONS publish [data on Construction output price indices](https://www.ons.gov.uk/businessindustryandtrade/constructionindustry/datasets/interimconstructionoutputpriceindices) which includes “Non-housing repair and maintenance” as well as “infrastructure” and “Public (other than housing” as potential matching categories.

The file cannot be imported using `read_excel()` (it results in a 403 error) so we have uploaded it to Google Drive (as *Construction Output Price Indices (OPIs), Quarter 3 (July to Sept) 2023*) and [published from there](https://docs.google.com/spreadsheets/d/1FFn0Vk31OMUZNFw8EMOa4brgOnEWsr6D/edit?usp=sharing&ouid=106245216815731294258&rtpof=true&sd=true) instead.

We've then created a pivot table - from the 'All construction' sheet - to calculate an average per year so that it matches our own timescales.

In [ ]:
#store the URL of the xlsx file
constructioncpiurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQVl5yZ-UCw5utMfwqoS-8ZMlJmqAQ8a8_pTLe8yyeZ6D-l34zjbieBJTDVBSg2eg/pub?gid=1424930737&single=true&output=csv"
#import that into a dataframe
constructioncpi_adjustdf = pd.read_csv(constructioncpiurl)
#show it
constructioncpi_adjustdf

,year_range,AVERAGE of All construction (new work and repair and maintenance) index 2015=100
0,201213,99.800
1,201314,99.400
2,201415,100.400
3,201516,102.700
4,201617,105.400
5,201718,108.400
6,201819,111.100
7,201920,112.200
8,202021,117.900
9,202122,128.600


### Merge the OPI data

As before, we get an error if we try to merge the data as it is, because the year_range field is an integer type in the CPI data.

We change that.

In [ ]:
#check data type
print(constructioncpi_adjustdf['year_range'].dtype)
#convert to str
constructioncpi_adjustdf['year_range'] = [str(i) for i in constructioncpi_adjustdf['year_range']]
#check data type
print(constructioncpi_adjustdf['year_range'].dtype)

int64
object


In [ ]:
constructioncpi_adjustdf.keys()

Index(['year_range', 'AVERAGE of All construction (new work and repair and maintenance) index 2015=100'], dtype='object')

In [ ]:
#merge the two dataframes on the year_range column and store in a new df
last5yrs_wdeflator = pd.merge(left = last5yrs_wdeflator,
                              right = constructioncpi_adjustdf,
                              on = 'year_range')

last5yrs_wdeflator.head()

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,...,measureclean,valuesclean,sitetypeclean,yearending,Financial Year,GDP Deflator.1,inflation_adjustedGDP,AVERAGE of CPI ALL ITEMS,inflation_adjustedCPI,AVERAGE of All construction (new work and repair and maintenance) index 2015=100
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092,0.000,133.000
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092,0.000,133.000
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,...,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092,0.000,133.000
3,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A01,ISLAND SITE,M13 9WL,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,General acute hospital,...,Cost to eradicate high risk backlog (GBP),0,General acute hospital,2023,2022-23,100.000,0.000,131.092,0.000,133.000
4,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,R0A06,UNIVERSITY DENTAL HOSPITAL OF MANCHESTER,M15 6FH,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,Cost to eradicate high risk backlog (GBP),0,Non inpatient,2023,2022-23,100.000,0.000,131.092,0.000,133.000


### Use the OPI figures to create a new column

And again, the OPI data needs to be multiplied by the current year's value (it is indexed to 2015).

This is in the last row.

In [ ]:
list(constructioncpi_adjustdf['AVERAGE of All construction (new work and repair and maintenance) index 2015=100'])[-1]

133.0

In [ ]:
#store the value for the current year
currentOPI = list(constructioncpi_adjustdf['AVERAGE of All construction (new work and repair and maintenance) index 2015=100'])[-1]
#create a new column based on multiplying one column by 1 divided by another
#multiplied by the current OPI
last5yrs_wdeflator['inflation_adjustedOPI'] = last5yrs_wdeflator['valuesclean']*(1/last5yrs_wdeflator['AVERAGE of All construction (new work and repair and maintenance) index 2015=100'])*currentOPI

### Export the CSV with the new column added

In [ ]:
#export as a CSV
last5yrs_wdeflator.to_csv('last5yrs_wdeflator.csv')
#download the file
files.download('last5yrs_wdeflator.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Analysis: change over time

Let's add some analysis. First, **high risk backlog** costs by commissioning region.

In [ ]:
pivotbyregionHIGHRISK = last5yrs_wdeflator[
    (last5yrs_wdeflator.measureclean == 'Cost to eradicate high risk backlog (GBP)')
    ].pivot_table(
    index="Commissioning Region",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

pivotbyregionHIGHRISK

year_range,201819,201920,202021,202122,202223
Commissioning Region,,,,,
EAST OF ENGLAND COMMISSIONING REGION,"99,183,682.169","245,231,364.848","261,439,102.858","258,903,305.124","292,944,001.000"
LONDON COMMISSIONING REGION,"712,099,826.310","873,018,258.824","785,598,523.859","835,022,805.622","935,014,807.000"
MIDLANDS COMMISSIONING REGION,"99,217,233.843","137,980,067.647","167,156,168.821","149,140,629.720","158,961,697.000"
NORTH EAST AND YORKSHIRE COMMISSIONING REGION,"96,711,829.883","169,658,976.105","187,429,856.302","275,838,577.784","520,291,401.000"
NORTH WEST COMMISSIONING REGION,"108,233,065.617","151,004,816.916","74,061,559.890","135,015,286.120","107,296,917.000"
SOUTH EAST COMMISSIONING REGION,"161,788,696.364","151,880,907.594","243,754,566.650","155,136,302.815","287,031,629.000"
SOUTH WEST COMMISSIONING REGION,"33,572,743.474","55,604,014.118","59,474,188.702","57,264,260.747","61,483,522.000"


## Analysis: by category of risk

Let's do another analysis, this time looking at **all** categories of risk, at a national level.



In [ ]:
# suppress scientific notation by setting float_format
pd.options.display.float_format = '£{:,.0f}'.format

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
pivotbyyear_category = last5yrs_wdeflator.pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "measureclean")

pivotbyyear_category

measureclean,Cost to eradicate high risk backlog (GBP),Cost to eradicate low risk backlog (GBP),Cost to eradicate moderate risk backlog (GBP),Cost to eradicate significant risk backlog (GBP)
year_range,,,,
201819,"£1,310,807,078","£1,141,392,753","£2,509,016,585","£2,769,646,085"
201920,"£1,784,378,406","£1,266,430,456","£3,873,168,090","£3,784,930,617"
202021,"£1,778,913,967","£1,312,794,953","£3,926,200,674","£3,395,870,680"
202122,"£1,866,321,168","£1,238,123,237","£3,892,909,079","£3,601,478,295"
202223,"£2,363,023,974","£1,318,445,706","£4,068,508,622","£3,887,993,248"


### Separate thousands with a comma

We can also format the numbers to use commas to separate thousands, and add a pound sign at the front.

In [ ]:
# change default formatting to add a pound sign, comma separators, and round to 0 decimals
pd.options.display.float_format = '£{:,.0f}'.format

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
last5yrs_wdeflator.pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "measureclean")

measureclean,Cost to eradicate high risk backlog (GBP),Cost to eradicate low risk backlog (GBP),Cost to eradicate moderate risk backlog (GBP),Cost to eradicate significant risk backlog (GBP)
year_range,,,,
201819,"£1,310,807,078","£1,141,392,753","£2,509,016,585","£2,769,646,085"
201920,"£1,784,378,406","£1,266,430,456","£3,873,168,090","£3,784,930,617"
202021,"£1,778,913,967","£1,312,794,953","£3,926,200,674","£3,395,870,680"
202122,"£1,866,321,168","£1,238,123,237","£3,892,909,079","£3,601,478,295"
202223,"£2,363,023,974","£1,318,445,706","£4,068,508,622","£3,887,993,248"


Now let's store that in a new dataframe

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
pivotByYear = last5yrs_wdeflator.pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "measureclean")

### Change label to reflect OPI inflation adjustment

And make sure that the column labels indicate that this is adjusted for inflation.

In [ ]:
#replace the closing bracket in the original column labels with ', OPI adj)'
#then replace the original column labels with the results
pivotByYear.columns = [i.replace(')',', OPI adj)') for i in list(pivotByYear.columns)]
pivotByYear

,"Cost to eradicate high risk backlog (GBP, OPI adj)","Cost to eradicate low risk backlog (GBP, OPI adj)","Cost to eradicate moderate risk backlog (GBP, OPI adj)","Cost to eradicate significant risk backlog (GBP, OPI adj)"
year_range,,,,
201819,"£1,310,807,078","£1,141,392,753","£2,509,016,585","£2,769,646,085"
201920,"£1,784,378,406","£1,266,430,456","£3,873,168,090","£3,784,930,617"
202021,"£1,778,913,967","£1,312,794,953","£3,926,200,674","£3,395,870,680"
202122,"£1,866,321,168","£1,238,123,237","£3,892,909,079","£3,601,478,295"
202223,"£2,363,023,974","£1,318,445,706","£4,068,508,622","£3,887,993,248"


Then export

In [ ]:
pivotByYear.to_csv("pivotByYear.csv")

### Show in millions

We can also format the figures so that they display as millions.

In [ ]:
# change formatting again to add 'm' at the end
pd.options.display.float_format = '£{:,.0f}m'.format

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
#round to -6 (millions)
#and divide by 1,000,000 to show the figures as millions
 #(otherwise it will have 6 zeroes)
round(last5yrs_wdeflator.pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "measureclean"),-6) / 1000000

measureclean,Cost to eradicate high risk backlog (GBP),Cost to eradicate low risk backlog (GBP),Cost to eradicate moderate risk backlog (GBP),Cost to eradicate significant risk backlog (GBP)
year_range,,,,
201819,"£1,311m","£1,141m","£2,509m","£2,770m"
201920,"£1,784m","£1,266m","£3,873m","£3,785m"
202021,"£1,779m","£1,313m","£3,926m","£3,396m"
202122,"£1,866m","£1,238m","£3,893m","£3,601m"
202223,"£2,363m","£1,318m","£4,069m","£3,888m"


In [ ]:
# change back!
pd.options.display.float_format = '{:.0f}'.format

### Show hospitals only

We can add a filter to just look at the costs to hospitals.

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
round(last5yrs_wdeflator[
     (last5yrs_wdeflator['sitetypeclean'] == 'General acute hospital')
    ].pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "measureclean"))

measureclean,Cost to eradicate high risk backlog (GBP),Cost to eradicate low risk backlog (GBP),Cost to eradicate moderate risk backlog (GBP),Cost to eradicate significant risk backlog (GBP)
year_range,,,,
201819,1211457853,845649187,2053926693,2355010586
201920,1593257463,854656112,3253566261,3320197075
202021,1508621851,933993841,3245013518,2851900496
202122,1553933379,897207754,3208142040,3087625172
202223,2078970702,958635137,3245030628,3226524970


In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
costs_hosp_only = round(last5yrs_wdeflator[
     (last5yrs_wdeflator['sitetypeclean'] == 'General acute hospital')
    ].pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "measureclean"))

costs_hosp_only.to_csv("costs_hosp_only.csv")

## Analysis: by hospital

In [ ]:
last5yrs_wdeflator.keys()

Index(['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type',
       'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board',
       'Local Authority', 'Site Type', 'measure', 'values', 'year_range',
       'Status', 'Tenure', 'Leased', 'Old Commissioning Region',
       'Leasehold Type', 'measureclean', 'valuesclean', 'sitetypeclean',
       'yearending', 'Financial Year', 'GDP Deflator.1',
       'inflation_adjustedGDP', 'AVERAGE of CPI ALL ITEMS',
       'inflation_adjustedCPI',
       'AVERAGE of All construction (new work and repair and maintenance) index 2015=100',
       'inflation_adjustedOPI'],
      dtype='object')

In [ ]:
last5yrs_wdeflator.measureclean.unique()

array(['Cost to eradicate high risk backlog (GBP)',
       'Cost to eradicate significant risk backlog (GBP)',
       'Cost to eradicate moderate risk backlog (GBP)',
       'Cost to eradicate low risk backlog (GBP)'], dtype=object)

In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyHospitalHIGHRISK = last5yrs_wdeflator[
    (last5yrs_wdeflator.measureclean == 'Cost to eradicate high risk backlog (GBP)') &
     (last5yrs_wdeflator['sitetypeclean'] == 'General acute hospital')
    ].pivot_table(
    index=["Site Name","Site Code","Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
pivotbyHospitalHIGHRISK = pivotbyHospitalHIGHRISK.sort_values(by=['202223'], ascending = False)

pivotbyHospitalHIGHRISK

,,year_range,201819,201920,202021,202122,202223
Site Name,Site Code,Commissioning Region,,,,,
AIREDALE GENERAL HOSPITAL,RCF22,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,4415574,16147390,37267542,35499829,335202292
CHARING CROSS HOSPITAL,RYJ02,LONDON COMMISSIONING REGION,167731185,166452153,166810839,160331693,173712630
ST MARY'S HOSPITAL,RYJ01,LONDON COMMISSIONING REGION,145091465,143985071,138714544,130676336,144830783
CROYDON UNIVERSITY HOSPITAL,RJ611,LONDON COMMISSIONING REGION,4066618,3493377,3394849,28216069,105738250
WYCOMBE HOSPITAL,RXQ50,SOUTH EAST COMMISSIONING REGION,6722447,5142667,95860850,34277492,84524165
...,...,...,...,...,...,...,...
UNIVERSITY HOSPITAL,RKB01,MIDLANDS COMMISSIONING REGION,0,0,0,0,NaN
WARDS 1 & 2 LONDON ROAD COMMUNITY HOSPITAL,RXMF4,MIDLANDS COMMISSIONING REGION,NaN,0,NaN,NaN,NaN
WEST CUMBERLAND HOSPITAL,RNLBX,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,2089656,NaN,NaN,NaN,NaN


### Cleaning hospital names/codes

That last query gives us 253 rows - but some hospitals appear more than once under different codes: BASILDON HOSPITAL (and various others) are in the data for two years with one site code, then changing to another.

More challenging is a 'NORTH MANCHESTER' that disappears when a NORTH MANCHESTER GENERAL HOSPITAL appears in the data with a different site code.

Running the same query with `"Site Code"` omitted gives us 13 fewer rows.






In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyHospitalHIGHRISK = last5yrs_wdeflator[
    (last5yrs_wdeflator.measureclean == 'Cost to eradicate high risk backlog (GBP)') &
     (last5yrs_wdeflator['sitetypeclean'] == 'General acute hospital')
    ].pivot_table(
    index=["Site Name","Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
pivotbyHospitalHIGHRISK = pivotbyHospitalHIGHRISK.sort_values(by=['202122'], ascending = False)

pivotbyHospitalHIGHRISK

,year_range,201819,201920,202021,202122,202223
Site Name,Commissioning Region,,,,,
CHARING CROSS HOSPITAL,LONDON COMMISSIONING REGION,167731185,166452153,166810839,160331693,173712630
ST MARY'S HOSPITAL,LONDON COMMISSIONING REGION,145091465,143985071,138714544,130676336,144830783
HAMMERSMITH HOSPITAL,LONDON COMMISSIONING REGION,76312586,75730665,NaN,70773856,74360488
THE QUEEN ELIZABETH HOSPITAL,EAST OF ENGLAND COMMISSIONING REGION,7749449,11407431,68998049,68121953,74175015
WEST SUFFOLK HOSPITAL,EAST OF ENGLAND COMMISSIONING REGION,3777532,96481027,86109901,63765950,65683727
...,...,...,...,...,...,...
THE ROYAL SUSSEX COUNTY HOSPITAL,SOUTH EAST COMMISSIONING REGION,1467225,2756320,3751137,NaN,NaN
WANSBECK GENERAL HOSPITAL,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,0,7112299,NaN,NaN,2520000
WARDS 1 & 2 LONDON ROAD COMMUNITY HOSPITAL,MIDLANDS COMMISSIONING REGION,NaN,0,NaN,NaN,NaN


Part of the explanation for this comes with some [googling around Basildon University Hospital and its codes]((https://www.cqc.org.uk/location/RDDH0), which shows that it changed trust, and then '[basildon change trust'](https://www.england.nhs.uk/publication/southend-university-hospital-nhs-foundation-trust/):

> "Mid and South Essex NHS Foundation Trust acquired Basildon and Thurrock University Hospitals NHS Foundation Trust and Mid Essex Hospital Services NHS Trust on 1 April 2020"

But there are also at least two hospitals which have the same name in different regions (PRINCESS ROYAL HOSPITAL and QUEEN ELIZABETH HOSPITAL).

### Adding the trust

We can add `'Trust Name'` to the query to reflect this. Now we get 268 rows.

In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyHospitalHIGHRISK = last5yrs_wdeflator[
    (last5yrs_wdeflator.measureclean == 'Cost to eradicate high risk backlog (GBP)') &
     (last5yrs_wdeflator['sitetypeclean'] == 'General acute hospital')
    ].pivot_table(
    index=["Site Name","Trust Code",'Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
pivotbyHospitalHIGHRISK = pivotbyHospitalHIGHRISK.sort_values(by=['202122'], ascending = False)

pivotbyHospitalHIGHRISK

,,,year_range,201819,201920,202021,202122,202223
Site Name,Trust Code,Trust Name,Commissioning Region,,,,,
CHARING CROSS HOSPITAL,RYJ,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,LONDON COMMISSIONING REGION,167731185,166452153,166810839,160331693,173712630
ST MARY'S HOSPITAL,RYJ,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,LONDON COMMISSIONING REGION,145091465,143985071,138714544,130676336,144830783
HAMMERSMITH HOSPITAL,RYJ,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,LONDON COMMISSIONING REGION,76312586,75730665,NaN,70773856,74360488
THE QUEEN ELIZABETH HOSPITAL,RCX,THE QUEEN ELIZABETH HOSPITAL KING'S LYNN NHS FOUNDATION TRUST,EAST OF ENGLAND COMMISSIONING REGION,7749449,11407431,68998049,68121953,74175015
WEST SUFFOLK HOSPITAL,RGR,WEST SUFFOLK NHS FOUNDATION TRUST,EAST OF ENGLAND COMMISSIONING REGION,3777532,96481027,86109901,63765950,65683727
...,...,...,...,...,...,...,...,...
WESTON GENERAL HOSPITAL,RA3,WESTON AREA HEALTH NHS TRUST,SOUTH WEST COMMISSIONING REGION,1134151,1970581,NaN,NaN,NaN
WESTON HOUSE,RP4,GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST,LONDON COMMISSIONING REGION,NaN,NaN,NaN,NaN,0
WORTHING HOSPITAL,RYR,WESTERN SUSSEX HOSPITALS NHS FOUNDATION TRUST,SOUTH EAST COMMISSIONING REGION,705916,414741,363104,NaN,NaN


In [ ]:
pivotbyHospitalHIGHRISK.to_csv('pivotbyHospitalHIGHRISK.csv')

### Repeating for all risk levels

And repeat for all risk levels

In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyHospitalALLRISK = last5yrs_wdeflator[
     (last5yrs_wdeflator['sitetypeclean'] == 'General acute hospital')
    ].pivot_table(
    index=["Site Name","Trust Code", 'Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
pivotbyHospitalALLRISK = pivotbyHospitalALLRISK.sort_values(by=['202223'], ascending = False)

pivotbyHospitalALLRISK

,,,year_range,201819,201920,202021,202122,202223
Site Name,Trust Code,Trust Name,Commissioning Region,,,,,
AIREDALE GENERAL HOSPITAL,RCF,AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,24001624,51745230,438306327,428063338,358310468
CHARING CROSS HOSPITAL,RYJ,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,LONDON COMMISSIONING REGION,377372500,364894053,369609650,355558356,349351889
LEIGHTON HOSPITAL,RBT,THE MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,53953819,438549404,259995929,286299548,337172814
ST THOMAS' HOSPITAL,RJ1,GUY'S AND ST THOMAS' NHS FOUNDATION TRUST,LONDON COMMISSIONING REGION,16597457,486551956,321695256,294922598,279398626
ST MARY'S HOSPITAL,RYJ,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,LONDON COMMISSIONING REGION,282014467,273038219,259617636,247637035,270410066
...,...,...,...,...,...,...,...,...
WEST CUMBERLAND HOSPITAL,RNL,NORTH CUMBRIA UNIVERSITY HOSPITALS NHS TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,23675797,NaN,NaN,NaN,NaN
WESTON GENERAL HOSPITAL,RA3,WESTON AREA HEALTH NHS TRUST,SOUTH WEST COMMISSIONING REGION,18210657,20698048,NaN,NaN,NaN
WHISTON HOSPITAL,RBN,ST HELENS AND KNOWSLEY TEACHING HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,0,NaN,NaN,0,NaN


## Analysis: by trust (all risks/sites)

Some exploration of that data in Excel shows a number of hospitals moving from one trust to another. Let's analyse by trust to see which trusts exist across all the years covered, and which new or old trusts only cover part of the timescale.

This time we remove the filters that narrowed down to hospitals and high risk repairs.

In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyTrustALLRISK = last5yrs_wdeflator.pivot_table(
    index=["Trust Name",'Trust Code',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
pivotbyTrustALLRISK = pivotbyTrustALLRISK.sort_values(by=['202122'], ascending = False)

pivotbyTrustALLRISK

,,year_range,201819,201920,202021,202122,202223
Trust Name,Trust Code,Commissioning Region,,,,,
IMPERIAL COLLEGE HEALTHCARE NHS TRUST,RYJ,LONDON COMMISSIONING REGION,827671401,796256154,786065137,761229709,769372189
GUY'S AND ST THOMAS' NHS FOUNDATION TRUST,RJ1,LONDON COMMISSIONING REGION,21871104,666790872,440453117,479190987,453214539
AIREDALE NHS FOUNDATION TRUST,RCF,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,24001624,51745230,438306327,428063338,358310468
NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,RX1,MIDLANDS COMMISSIONING REGION,157175121,151614369,459477034,421246829,437530919
BARTS HEALTH NHS TRUST,R1H,LONDON COMMISSIONING REGION,238965985,316440975,200410783,325857558,338329666
...,...,...,...,...,...,...,...
WESTERN SUSSEX HOSPITALS NHS FOUNDATION TRUST,RYR,SOUTH EAST COMMISSIONING REGION,27984933,24868022,21457545,NaN,NaN
WESTON AREA HEALTH NHS TRUST,RA3,SOUTH WEST COMMISSIONING REGION,18509937,20994394,NaN,NaN,NaN
WIRRAL COMMUNITY NHS FOUNDATION TRUST,RY7,NORTH WEST COMMISSIONING REGION,128092,167139,NaN,NaN,NaN


In [ ]:
pivotbyTrustALLRISK.to_csv('pivotbyTrustALLRISK.csv')

## Add in codes for trusts that have changed

Many trusts only have figures for some years, either because they are new, or because they are older trusts that ceased to exist (changing name, or becoming part of a new or existing trust).

Some trusts have figures for all the years, but during that time acquired other trusts whose historical costs need to be factored in.

For example: in 2023 SOUTHPORT AND ORMSKIRK HOSPITAL NHS TRUST is going to be succeeded by the code RBN. RBN is the code for ST HELENS AND KNOWSLEY TEACHING HOSPITALS NHS TRUST, which has had quite low figures for the last 4 years (dropping from £957,207 to £129,277). Now, next year it's going to include the figures for Southport - the most recent of which was £54m.
So St Helens's figures will jump from £129k to £54m - *unless* we add in the historical figures of Southport to more accurately reflect the fact that costs are reported under just one code which previously were reported under two.

NHS Digital [publishes data on 'Successor Organisations'](https://digital.nhs.uk/services/organisation-data-service/export-data-files/csv-downloads/miscellaneous).

We've downloaded and unzipped the succarc.csv (Archived Successor Organisations) and succ.csv (Successor Organisations) files from that page and published as a Google Sheet, adding headings so that we can import and merge with our data here.

We need to import that and merge it with our existing data so we can pivot on the most recent codes.

***NOTE: Some old trust codes have multiple successor codes. RW6 Pennine Acute Hospitals NHS Trust) is mapped to both NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST and MANCHESTER UNIVERSITY NHS FOUNDATION TRUST. Joining the old codes with the new ones will create two rows where one existed before (one for each new code). In this scenario and others such as MERSEY CARE NHS FOUNDATION TRUST and LANCASHIRE & SOUTH CUMBRIA NHS FOUNDATION TRUST (both successor trusts to RVT) manual cleaning needs to be undertaken to allocate each hospital to its new parent trust.***

In this case, in the spreadsheet exported at the end of this notebook **North Manchester Hospital has been allocated solely to Manchester University NHS Foundation Trust, and Rochdale, Royal Oldham, Birch Hill and Fairfield Hospital to Northern Care**, by deleting rows where those sites were allocated to the other trust.


In [ ]:
#store the URL
succarcurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRgtsnTAuRKfLgjwMzz_DdFDcxBF8t3KqX30HX9vKJZDeaMq2p-AZD-1dBAboOQwUHYwNaWkUwY56SG/pub?gid=409838561&single=true&output=csv"
#import
succarcdf = pd.read_csv(succarcurl)


In [ ]:
succarcdf.keys()

Index(['Trust Code', 'Most Recent Trust Code', 'Most Recent Trust Name',
       'Notes on trust data', 'Source', 'Count'],
      dtype='object')

In [ ]:
#merge the two dataframes on the year_range column and store in a new df
last5yrs_wdeflator = pd.merge(left = last5yrs_wdeflator,
                              right = succarcdf,
                              on = 'Trust Code')

last5yrs_wdeflator.head(3)

,Trust Code,Trust Name,Commissioning Region,Trust Type,Site Code,Site Name,Post Code,Integrated Care Board,Local Authority,Site Type,...,inflation_adjustedGDP,AVERAGE of CPI ALL ITEMS,inflation_adjustedCPI,AVERAGE of All construction (new work and repair and maintenance) index 2015=100,inflation_adjustedOPI,Most Recent Trust Code,Most Recent Trust Name,Notes on trust data,Source,Count
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I0Z5L,"MEDICAL RECORD 2, ASHBURY'S BUSINESS CENTRE",M12 5BX,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,0,131,0,133,0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,ERIC data,1
1,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,I5U0V,"MEDICAL RECORDS 1, DELTA INDUSTRIAL ESTATE",M18 8DB,NHS GREATER MANCHESTER ICB,MANCHESTER CITY COUNCIL,Non inpatient,...,0,131,0,133,0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,ERIC data,1
2,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,ACUTE - TEACHING,O3L2I,MEDWAY HEALTH CENTRE,M33 4PS,NHS GREATER MANCHESTER ICB,TRAFFORD METROPOLITAN BOROUGH COUNCIL,Non inpatient,...,0,131,0,133,0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,ERIC data,1


### Clean trusts for site data tables too

We need to do this for the hospital-level analysis, too.

Because the data frame was generated by a pivot table, the indices are not recognised as keys and will not be retained in a merge.

To fix this we just need to add `.reset_index()` to the pivot table dataframe.

In [ ]:
#the pivot table dataframe doesn't have the indices as keys
pivotbyHospitalALLRISK.keys()

Index(['201819', '201920', '202021', '202122', '202223'], dtype='object', name='year_range')

In [ ]:
#add .reset_index to bring the index fields in as columns too
pivotbyHospitalALLRISK_merged = pd.merge(left = pivotbyHospitalALLRISK.reset_index(),
                              right = succarcdf,
                              how='left',
                              on = 'Trust Code')
#reassign to the original data frame
pivotbyHospitalALLRISK = pivotbyHospitalALLRISK_merged
#check the keys
pivotbyHospitalALLRISK.keys()

Index(['Site Name', 'Trust Code', 'Trust Name', 'Commissioning Region',
       '201819', '201920', '202021', '202122', '202223',
       'Most Recent Trust Code', 'Most Recent Trust Name',
       'Notes on trust data', 'Source', 'Count'],
      dtype='object')

In [ ]:
#add .reset_index to bring the index fields in as columns too
pivotbyHospitalHIGHRISK_merged = pd.merge(left = pivotbyHospitalHIGHRISK.reset_index(),
                              right = succarcdf,
                              how='left',
                              on = 'Trust Code')
#reassign to the original data frame
pivotbyHospitalHIGHRISK = pivotbyHospitalHIGHRISK_merged
#check the keys
pivotbyHospitalHIGHRISK.keys()

Index(['Site Name', 'Trust Code', 'Trust Name', 'Commissioning Region',
       '201819', '201920', '202021', '202122', '202223',
       'Most Recent Trust Code', 'Most Recent Trust Name',
       'Notes on trust data', 'Source', 'Count'],
      dtype='object')

### Clean North Manchester

We have a particular problem in that the site North Manchester (hospital) has a Trust Code which has two successor trust codes. That creates two rows assigning it to both, so it will be double-counted.

Most hospitals from Pennine Acute Hospitals NHS Trust joined the Northern Care Alliance, but North Manchester joined Manchester University NHS Foundation Trust.

We need to remove the row connecting North Manchester to the Northern Care Alliance.

In [ ]:
#show the rows in question
pivotbyHospitalHIGHRISK[pivotbyHospitalHIGHRISK['Site Name'].str.contains('NORTH MANCHESTER')]


,Site Name,Trust Code,Trust Name,Commissioning Region,201819,201920,202021,202122,202223,Most Recent Trust Code,Most Recent Trust Name,Notes on trust data,Source,Count
25,NORTH MANCHESTER GENERAL HOSPITAL,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,NaN,NaN,NaN,14013608,7561159,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,ERIC data,1
248,NORTH MANCHESTER,RW6,PENNINE ACUTE HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,10312288,11047276,8195697,NaN,NaN,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,Data for trust code RW6 in earlier years inclu...,succarc.csv,2
249,NORTH MANCHESTER,RW6,PENNINE ACUTE HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,10312288,11047276,8195697,NaN,NaN,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Data for trust code RW6 in earlier years inclu...,succarc.csv,2


The code below is generated by Google Bard in response to this query:

> I have a dataframe called pivotbyHospitalHIGHRISK
Write some Python which removes the row where the field 'Site Name' contains 'NORTH MANCHESTER' and the 'Most Recent Trust Name' field contains 'NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST'

Comments have been added and the code adapted.

In [ ]:
#code below generated by Google Bard
#define a function which takes a dataframe and calls it df
def removehospitals_NCANFT(df):
  #return a dataframe which has all rows apart from those which contain the specified values in the specified fields
    return df.loc[(df['Site Name'] != 'NORTH MANCHESTER') | (df['Most Recent Trust Name'] != 'NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST')]

#run the function on our dataframe
pivotbyHospitalHIGHRISK_minusMCR = removehospitals_NCANFT(pivotbyHospitalHIGHRISK.copy())
#check the lengths
print(len(pivotbyHospitalHIGHRISK))
print(len(pivotbyHospitalHIGHRISK_minusMCR))

290
289


In [ ]:
#check the rows
pivotbyHospitalHIGHRISK_minusMCR[pivotbyHospitalHIGHRISK_minusMCR['Site Name'].str.contains('NORTH MANCHESTER')]

,Site Name,Trust Code,Trust Name,Commissioning Region,201819,201920,202021,202122,202223,Most Recent Trust Code,Most Recent Trust Name,Notes on trust data,Source,Count
25,NORTH MANCHESTER GENERAL HOSPITAL,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,NaN,NaN,NaN,14013608,7561159,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,ERIC data,1
249,NORTH MANCHESTER,RW6,PENNINE ACUTE HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,10312288,11047276,8195697,NaN,NaN,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Data for trust code RW6 in earlier years inclu...,succarc.csv,2


We now need to repeat the process for the hospitals which are in Northern Care but not Manchester University NHS Trust

In [ ]:
#check the rows
pivotbyHospitalHIGHRISK_minusMCR[pivotbyHospitalHIGHRISK_minusMCR['Site Name'].str.contains('OLDHAM')]

,Site Name,Trust Code,Trust Name,Commissioning Region,201819,201920,202021,202122,202223,Most Recent Trust Code,Most Recent Trust Name,Notes on trust data,Source,Count
77,ROYAL OLDHAM HOSPITAL,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,NaN,NaN,NaN,4017558,1898806,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,NaN,ERIC data,1
276,THE ROYAL OLDHAM HOSPITAL,RW6,PENNINE ACUTE HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,1463479,3491287,1214627,NaN,NaN,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,Data for trust code RW6 in earlier years inclu...,succarc.csv,2
277,THE ROYAL OLDHAM HOSPITAL,RW6,PENNINE ACUTE HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,1463479,3491287,1214627,NaN,NaN,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Data for trust code RW6 in earlier years inclu...,succarc.csv,2


In [ ]:
#code below generated by Google Bard
#define a function which takes a dataframe and calls it df
def removehospitals_MUNFT(df):
  #remove all rows apart from those which contain the specified values in the specified fields
  oldhameremoved = df.loc[(df['Site Name'] != 'THE ROYAL OLDHAM HOSPITAL') | (df['Most Recent Trust Name'] != 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST')]
  rochdaleremoved = oldhameremoved.loc[(oldhameremoved['Site Name'] != 'ROCHDALE INFIRMARY') | (oldhameremoved['Most Recent Trust Name'] != 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST')]
  fairfieldremoved = rochdaleremoved.loc[(rochdaleremoved['Site Name'] != 'FAIRFIELD GENERAL HOSPITAL') | (rochdaleremoved['Most Recent Trust Name'] != 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST')]
  birchremoved = fairfieldremoved.loc[(fairfieldremoved['Site Name'] != 'BIRCH HILL HOSPITAL') | (fairfieldremoved['Most Recent Trust Name'] != 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST')]
  #return the results
  return(birchremoved)

#show length before
print(len(pivotbyHospitalHIGHRISK_minusMCR))
#run the function on our dataframe
pivotbyHospitalHIGHRISK_minusMCR = removehospitals_MUNFT(pivotbyHospitalHIGHRISK_minusMCR.copy())
#check the length after
print(len(pivotbyHospitalHIGHRISK_minusMCR))

289
285


In [ ]:
#check the rows
pivotbyHospitalHIGHRISK_minusMCR[pivotbyHospitalHIGHRISK_minusMCR['Site Name'].str.contains('OLDHAM')]

,Site Name,Trust Code,Trust Name,Commissioning Region,201819,201920,202021,202122,202223,Most Recent Trust Code,Most Recent Trust Name,Notes on trust data,Source,Count
77,ROYAL OLDHAM HOSPITAL,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,NaN,NaN,NaN,4017558,1898806,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,NaN,ERIC data,1
276,THE ROYAL OLDHAM HOSPITAL,RW6,PENNINE ACUTE HOSPITALS NHS TRUST,NORTH WEST COMMISSIONING REGION,1463479,3491287,1214627,NaN,NaN,RM3,NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST,Data for trust code RW6 in earlier years inclu...,succarc.csv,2


Now we can overwrite the original data frame.

In [ ]:
pivotbyHospitalHIGHRISK = pivotbyHospitalHIGHRISK_minusMCR

And repeat for all risk.

In [ ]:
#run the function on our dataframe
pivotbyHospitalALLRISK_minusMCR = removehospitals_NCANFT(pivotbyHospitalALLRISK.copy())
#check the lengths
print(len(pivotbyHospitalALLRISK))
print(len(pivotbyHospitalALLRISK_minusMCR))
#run the second function on our dataframe
pivotbyHospitalALLRISK_minusMCR = removehospitals_MUNFT(pivotbyHospitalALLRISK_minusMCR.copy())
#check the length after
print(len(pivotbyHospitalALLRISK_minusMCR))

290
289
285


In [ ]:
#overwrite the original
pivotbyHospitalALLRISK = pivotbyHospitalALLRISK_minusMCR

### Analysis: by 'most recent' trust code

We can now adapt our last pivot table to focus on the new 'most recent trust code'.

In [ ]:
last5yrs_wdeflator.keys()

Index(['Trust Code', 'Trust Name', 'Commissioning Region', 'Trust Type',
       'Site Code', 'Site Name', 'Post Code', 'Integrated Care Board',
       'Local Authority', 'Site Type', 'measure', 'values', 'year_range',
       'Status', 'Tenure', 'Leased', 'Old Commissioning Region',
       'Leasehold Type', 'measureclean', 'valuesclean', 'sitetypeclean',
       'yearending', 'Financial Year', 'GDP Deflator.1',
       'inflation_adjustedGDP', 'AVERAGE of CPI ALL ITEMS',
       'inflation_adjustedCPI',
       'AVERAGE of All construction (new work and repair and maintenance) index 2015=100',
       'inflation_adjustedOPI', 'Most Recent Trust Code',
       'Most Recent Trust Name', 'Notes on trust data', 'Source', 'Count'],
      dtype='object')

In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyTrustALLRISK = last5yrs_wdeflator.pivot_table(
    index=['Most Recent Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
#pivotbyTrustALLRISK = pivotbyTrustALLRISK.sort_values(by=['202122'], ascending = False)

pivotbyTrustALLRISK.head(3)

,year_range,201819,201920,202021,202122,202223
Most Recent Trust Name,Commissioning Region,,,,,
AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,24001624,51745230,438306327,428063338,358310468
ALDER HEY CHILDRENS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,16244914,16085651,15307973,2621972,1458000
ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATION TRUST,SOUTH EAST COMMISSIONING REGION,18150306,17343324,17945935,79299330,78942798


### Add notes for changed trusts

But we would like to know where the most recent name might include other names in the past.

We start by pivoting to create a table with a row for each trust name, old or new.

In [ ]:
#pivot by site name, with a column for each year
#two filters
#summing the inflation adjusted figures
pivotbyNewOldTrustALLRISK = last5yrs_wdeflator.pivot_table(
    index=['Most Recent Trust Name','Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

#sort it
#pivotbyTrustALLRISK = pivotbyTrustALLRISK.sort_values(by=['202122'], ascending = False)

pivotbyNewOldTrustALLRISK.head(3)

,,year_range,201819,201920,202021,202122,202223
Most Recent Trust Name,Trust Name,Commissioning Region,,,,,
AIREDALE NHS FOUNDATION TRUST,AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,24001624,51745230,438306327,428063338,358310468
ALDER HEY CHILDRENS NHS FOUNDATION TRUST,ALDER HEY CHILDRENS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,16244914,16085651,15307973,2621972,1458000
ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATION TRUST,ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATION TRUST,SOUTH EAST COMMISSIONING REGION,18150306,17343324,17945935,79299330,78942798


In [ ]:
pivotbyNewOldTrustALLRISK.to_csv('pivotbyNewOldTrustALLRISK.csv')

Once exported we can work out in Excel where a new trust appears more than once because it has more than one (old) trust connected with it.

We then group those and add notes which explain where a trust includes previous trusts' figures.

That is used to create a lookup table which we import back into the notebook here.

In [ ]:
#store the URL
namenotesurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRgtsnTAuRKfLgjwMzz_DdFDcxBF8t3KqX30HX9vKJZDeaMq2p-AZD-1dBAboOQwUHYwNaWkUwY56SG/pub?gid=458095730&single=true&output=csv"
#import
namenotes = pd.read_csv(namenotesurl)

#the lines below are commented out as they resulted in the removal of some trusts
#which left the overall total short of the figure it should be
#instead we add it to the analysis XLSX

#merge the two dataframes on the year_range column and store in a new df
#last5yrs_wdeflator = pd.merge(left = last5yrs_wdeflator,
   #                           right = namenotes,
     #                         on = 'Most Recent Trust Name')

#last5yrs_wdeflator.head()

In [ ]:
namenotes.to_csv("namenotes.csv")

### Create FUNCTION to calculate YoY change

We can calculate a year on year change by subtracting the second-to-last column from the last column (the most recent year); then to get that as a percentage repeating the process to divide the new last column (the change) by the third-to-last column (the last year's figure).

When exported there's a potential problem with percentage figures not rendering correctly, so we create a duplicate column where the figure is multiplied by 100 to provide an alternative (e.g. 0.5 would become '50' percent).

In [ ]:
#define a function which takes a dataframe
def addyoycols(df):
  #subtract the second to last column from the last column
  df['YoY change'] = df[df.keys()[-1]] - df[df.keys()[-2]]
  #divide that new column by what is now the third to last column
  df['YoY change %'] = df[df.keys()[-1]] / df[df.keys()[-3]]
  #create a duplicate column where the figure is multiplied by 100 to provide an alternative (e.g. 0.5 would become '50' percent).
  df['YoY change %aswholenum'] = df['YoY change %'] * 100
  #return the resulting dataframe to whatever called the function
  return(df)

In [ ]:
#assign the results of the function to the dataframe (overwrite it)
pivotbyTrustALLRISK = addyoycols(pivotbyTrustALLRISK)

In [ ]:
pivotbyTrustALLRISK.head(3)

,year_range,201819,201920,202021,202122,202223,YoY change,YoY change %,YoY change %aswholenum
Most Recent Trust Name,Commissioning Region,,,,,,,,
AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,24001624,51745230,438306327,428063338,358310468,-69752870,-0,-16
ALDER HEY CHILDRENS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,16244914,16085651,15307973,2621972,1458000,-1163972,-0,-44
ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATION TRUST,SOUTH EAST COMMISSIONING REGION,18150306,17343324,17945935,79299330,78942798,-356532,-0,-0


### Export again...

In [ ]:
#export as a CSV
last5yrs_wdeflator.to_csv('last5yrs_wdeflator.csv')
pivotbyTrustALLRISK.to_csv('pivotbyTrustALLRISK.csv')

#download the file
files.download('last5yrs_wdeflator.csv')
files.download('pivotbyTrustALLRISK.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Analysis: by (recent) trust and risk type

Let's break down the results further by level of risk.

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
last5yrs_wdeflator.pivot_table(
    index=["measureclean",'Most Recent Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

year_range                                                                                                                                          201819  \
measureclean                                     Most Recent Trust Name                             Commissioning Region                                     
Cost to eradicate high risk backlog (GBP)        AIREDALE NHS FOUNDATION TRUST                      NORTH EAST AND YORKSHIRE COMMISSIONING REGION  4415574   
                                                 ALDER HEY CHILDRENS NHS FOUNDATION TRUST           NORTH WEST COMMISSIONING REGION                  47885   
                                                 ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATIO... SOUTH EAST COMMISSIONING REGION                1142800   
                                                 AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NH... SOUTH WEST COMMISSIONING REGION                2077602   
                                                 BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSP... LONDON COMMISSIONING REGION                     215482   
...                                                                                                                                                    ...   
Cost to eradicate significant risk backlog (GBP) WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST NORTH WEST COMMISSIONING REGION                4194875   
                                                 WYE VALLEY NHS TRUST                               MIDLANDS COMMISSIONING REGION                  1767978   
                                                 YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST      SOUTH WEST COMMISSIONING REGION                6322540   
                                                 YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOU... NORTH EAST AND YORKSHIRE COMMISSIONING REGION 15998287   
                                                 YORKSHIRE AMBULANCE SERVICE NHS TRUST              NORTH EAST AND YORKSHIRE COMMISSIONING REGION  1281198   

year_range                                                                                                                                          201920  \
measureclean                                     Most Recent Trust Name                             Commissioning Region                                     
Cost to eradicate high risk backlog (GBP)        AIREDALE NHS FOUNDATION TRUST                      NORTH EAST AND YORKSHIRE COMMISSIONING REGION 16147390   
                                                 ALDER HEY CHILDRENS NHS FOUNDATION TRUST           NORTH WEST COMMISSIONING REGION                  47415   
                                                 ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATIO... SOUTH EAST COMMISSIONING REGION                1091988   
                                                 AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP NH... SOUTH WEST COMMISSIONING REGION                 145161   
                                                 BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSP... LONDON COMMISSIONING REGION                          0   
...                                                                                                                                                    ...   
Cost to eradicate significant risk backlog (GBP) WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST NORTH WEST COMMISSIONING REGION                4167519   
                                                 WYE VALLEY NHS TRUST                               MIDLANDS COMMISSIONING REGION                  1202479   
                                                 YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST      SOUTH WEST COMMISSIONING REGION                6260554   
                                                 YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOU... NORTH EAST AND YORKSHIRE COMMISSIONING REGION 15517174   
                                                 

### Repeating rows (risk level)

Currently the risk level isn't repeated - but [we can fix that](https://stackoverflow.com/questions/47904813/pandas-pivot-repeat-index-row) by adding `.reset_index()`

In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
last5yrs_wdeflator.pivot_table(
    index=["measureclean",'Most Recent Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range").reset_index()

year_range,measureclean,Most Recent Trust Name,Commissioning Region,201819,201920,202021,202122,202223
0,Cost to eradicate high risk backlog (GBP),AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,4415574,16147390,37267542,35499829,335202292
1,Cost to eradicate high risk backlog (GBP),ALDER HEY CHILDRENS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,47885,47415,45123,914763,728000
2,Cost to eradicate high risk backlog (GBP),ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATI...,SOUTH EAST COMMISSIONING REGION,1142800,1091988,1126080,21108,270354
3,Cost to eradicate high risk backlog (GBP),AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP N...,SOUTH WEST COMMISSIONING REGION,2077602,145161,63223,0,0
4,Cost to eradicate high risk backlog (GBP),"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",LONDON COMMISSIONING REGION,215482,0,0,0,400000
...,...,...,...,...,...,...,...,...
847,Cost to eradicate significant risk backlog (GBP),"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",NORTH WEST COMMISSIONING REGION,4194875,4167519,18187920,18244928,18601752
848,Cost to eradicate significant risk backlog (GBP),WYE VALLEY NHS TRUST,MIDLANDS COMMISSIONING REGION,1767978,1202479,1898400,1191721,1648344
849,Cost to eradicate significant risk backlog (GBP),YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,SOUTH WEST COMMISSIONING REGION,6322540,6260554,6136618,6243547,5902272
850,Cost to eradicate significant risk backlog (GBP),YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FO...,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,15998287,15517174,14766980,12011753,11614372


### Add YoY cols (run function)

Now we can reuse that function from earlier



In [ ]:
#pivot by year_range, with a column for each measure
#summing the inflation adjusted figures
pivotbyTrustRISKDETAIL = last5yrs_wdeflator.pivot_table(
    index=["measureclean",'Most Recent Trust Name',"Commissioning Region"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range").reset_index()

#assign the results of the function to the dataframe (overwrite it)
pivotbyTrustRISKDETAIL = addyoycols(pivotbyTrustRISKDETAIL)

pivotbyTrustRISKDETAIL

year_range,measureclean,Most Recent Trust Name,Commissioning Region,201819,201920,202021,202122,202223,YoY change,YoY change %,YoY change %aswholenum
0,Cost to eradicate high risk backlog (GBP),AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,4415574,16147390,37267542,35499829,335202292,299702463,8,844
1,Cost to eradicate high risk backlog (GBP),ALDER HEY CHILDRENS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,47885,47415,45123,914763,728000,-186763,-0,-20
2,Cost to eradicate high risk backlog (GBP),ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATI...,SOUTH EAST COMMISSIONING REGION,1142800,1091988,1126080,21108,270354,249246,12,1181
3,Cost to eradicate high risk backlog (GBP),AVON AND WILTSHIRE MENTAL HEALTH PARTNERSHIP N...,SOUTH WEST COMMISSIONING REGION,2077602,145161,63223,0,0,0,NaN,NaN
4,Cost to eradicate high risk backlog (GBP),"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",LONDON COMMISSIONING REGION,215482,0,0,0,400000,400000,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...
847,Cost to eradicate significant risk backlog (GBP),"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",NORTH WEST COMMISSIONING REGION,4194875,4167519,18187920,18244928,18601752,356824,0,2
848,Cost to eradicate significant risk backlog (GBP),WYE VALLEY NHS TRUST,MIDLANDS COMMISSIONING REGION,1767978,1202479,1898400,1191721,1648344,456623,0,38
849,Cost to eradicate significant risk backlog (GBP),YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,SOUTH WEST COMMISSIONING REGION,6322540,6260554,6136618,6243547,5902272,-341275,-0,-5
850,Cost to eradicate significant risk backlog (GBP),YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FO...,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,15998287,15517174,14766980,12011753,11614372,-397381,-0,-3


### Add Notes back in

We tried to include notes in the pivot like this: `index=["measureclean",'Most Recent Trust Name',"Commissioning Region","NOTES"]`

However, that resulted in a table *only* showing those trusts with notes.

So we need to add those notes *after* the pivot.

In [ ]:
pivotbyTrustRISKDETAIL.keys()

Index(['measureclean', 'Most Recent Trust Name', 'Commissioning Region',
       '201819', '201920', '202021', '202122', '202223', 'YoY change',
       'YoY change %', 'YoY change %aswholenum'],
      dtype='object', name='year_range')

In [ ]:
#store the URL
namenotesurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRgtsnTAuRKfLgjwMzz_DdFDcxBF8t3KqX30HX9vKJZDeaMq2p-AZD-1dBAboOQwUHYwNaWkUwY56SG/pub?gid=458095730&single=true&output=csv"
#import
namenotes = pd.read_csv(namenotesurl)

#merge the two dataframes on the year_range column and store in a new df
#pivotbyTrustRISKDETAIL = pd.merge(left = pivotbyTrustRISKDETAIL,
   #                           right = namenotes,
      #                        on = 'Most Recent Trust Name')

pivotbyTrustRISKDETAIL.head(3)

year_range,measureclean,Most Recent Trust Name,Commissioning Region,201819,201920,202021,202122,202223,YoY change,YoY change %,YoY change %aswholenum
0,Cost to eradicate high risk backlog (GBP),AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,4415574,16147390,37267542,35499829,335202292,299702463,8,844
1,Cost to eradicate high risk backlog (GBP),ALDER HEY CHILDRENS NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,47885,47415,45123,914763,728000,-186763,-0,-20
2,Cost to eradicate high risk backlog (GBP),ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATI...,SOUTH EAST COMMISSIONING REGION,1142800,1091988,1126080,21108,270354,249246,12,1181


### Export

In [ ]:
#export as a CSV - remove index
pivotbyTrustRISKDETAIL.to_csv('pivotbyTrustRISKDETAIL.csv', index=False)
#download the file
files.download('pivotbyTrustRISKDETAIL.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Repeat trust analysis for acute hospitals only

We want to just focus on acute hospitals, so let's adapt the previous code to filter on that.

In [ ]:
#pivot by year_range, with a column for each measure
#we add sitetypeclean so that's explicit
#summing the inflation adjusted figures
pivotbyTrust_GACONLY_RISK = last5yrs_wdeflator[last5yrs_wdeflator.sitetypeclean == 'General acute hospital'].pivot_table(
    index=["measureclean",'Most Recent Trust Name',"Commissioning Region","sitetypeclean"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range").reset_index()

#assign the results of the function to the dataframe (overwrite it)
pivotbyTrust_GACONLY_RISK = addyoycols(pivotbyTrust_GACONLY_RISK)

pivotbyTrust_GACONLY_RISK.head(3)

year_range,measureclean,Most Recent Trust Name,Commissioning Region,sitetypeclean,201819,201920,202021,202122,202223,YoY change,YoY change %,YoY change %aswholenum
0,Cost to eradicate high risk backlog (GBP),AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,General acute hospital,4415574,16147390,37267542,35499829,335202292,299702463,8,844
1,Cost to eradicate high risk backlog (GBP),ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATI...,SOUTH EAST COMMISSIONING REGION,General acute hospital,1142800,1091988,NaN,21108,270354,249246,12,1181
2,Cost to eradicate high risk backlog (GBP),"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",LONDON COMMISSIONING REGION,General acute hospital,215482,0,0,0,NaN,NaN,NaN,NaN


### Add in hospitals for later cleaning

We are going to add in the hospitals here so that those can be manually removed where they're allocated to more than one trust (see note above).

In [ ]:
pivotbyTrust_GACONLY_RISKwHOSP = last5yrs_wdeflator[last5yrs_wdeflator.sitetypeclean == 'General acute hospital'].pivot_table(
    index=["measureclean",'Site Name','Most Recent Trust Name',"Commissioning Region","sitetypeclean"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range").reset_index()

#assign the results of the function to the dataframe (overwrite it)
pivotbyTrust_GACONLY_RISKwHOSP = addyoycols(pivotbyTrust_GACONLY_RISKwHOSP)

pivotbyTrust_GACONLY_RISKwHOSP.head(3)

year_range,measureclean,Site Name,Most Recent Trust Name,Commissioning Region,sitetypeclean,201819,201920,202021,202122,202223,YoY change,YoY change %,YoY change %aswholenum
0,Cost to eradicate high risk backlog (GBP),ADDENBROOKE'S HOSPITAL,CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION ...,EAST OF ENGLAND COMMISSIONING REGION,General acute hospital,36647055,48359239,29979611,24774377,62370318,37595941,2,152
1,Cost to eradicate high risk backlog (GBP),AIREDALE GENERAL HOSPITAL,AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,General acute hospital,4415574,16147390,37267542,35499829,335202292,299702463,8,844
2,Cost to eradicate high risk backlog (GBP),ANDOVER WAR MEMORIAL HOSPITAL,HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST,SOUTH EAST COMMISSIONING REGION,General acute hospital,31724,31413,18613,NaN,NaN,NaN,NaN,NaN


### Clean Manchester

We re-run the code from earlier to remove double counted hospital sites.

In [ ]:

#run the function on our dataframe
pivotbyTrust_GACONLY_RISKwHOSP_minusMCR = removehospitals_NCANFT(pivotbyTrust_GACONLY_RISKwHOSP.copy())
#check the lengths
print(len(pivotbyTrust_GACONLY_RISKwHOSP))
print(len(pivotbyTrust_GACONLY_RISKwHOSP_minusMCR))

#run the second function on our dataframe
pivotbyTrust_GACONLY_RISKwHOSP_minusMCR = removehospitals_MUNFT(pivotbyTrust_GACONLY_RISKwHOSP_minusMCR.copy())
#check the lengths
print(len(pivotbyTrust_GACONLY_RISKwHOSP_minusMCR))



1020
1016
1000


In [ ]:
#and re-assign to the original df
pivotbyTrust_GACONLY_RISKwHOSP = pivotbyTrust_GACONLY_RISKwHOSP_minusMCR

In [ ]:
#export as a CSV - remove index
pivotbyTrust_GACONLY_RISK.to_csv('pivotbyTrust_GACONLY_RISK.csv', index=False)
#download the file
#files.download('pivotbyTrust_GACONLY_RISK.csv')

pivotbyTrust_GACONLY_RISKwHOSP.to_csv('pivotbyTrust_GACONLY_RISKwHOSP.csv', index=False)

## General acute hospitals - but not most recent trust

Combining trusts addresses some potential problems but can introduce new ones, so we also calculate without that.

In [ ]:
#pivot by year_range, with a column for each measure
#we add sitetypeclean so that's explicit
#summing the inflation adjusted figures
pivotbyTrustOLD_GACONLY_RISK = last5yrs_wdeflator[last5yrs_wdeflator.sitetypeclean == 'General acute hospital'].pivot_table(
    index=["measureclean",'Trust Name',"Commissioning Region","sitetypeclean"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range").reset_index()

#assign the results of the function to the dataframe (overwrite it)
pivotbyTrustOLD_GACONLY_RISK = addyoycols(pivotbyTrustOLD_GACONLY_RISK)

pivotbyTrustOLD_GACONLY_RISK.head(3)

year_range,measureclean,Trust Name,Commissioning Region,sitetypeclean,201819,201920,202021,202122,202223,YoY change,YoY change %,YoY change %aswholenum
0,Cost to eradicate high risk backlog (GBP),AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,NORTH WEST COMMISSIONING REGION,General acute hospital,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cost to eradicate high risk backlog (GBP),AIREDALE NHS FOUNDATION TRUST,NORTH EAST AND YORKSHIRE COMMISSIONING REGION,General acute hospital,4415574,16147390,37267542,35499829,335202292,299702463,8,844
2,Cost to eradicate high risk backlog (GBP),ASHFORD AND ST. PETER'S HOSPITALS NHS FOUNDATI...,SOUTH EAST COMMISSIONING REGION,General acute hospital,1142800,1091988,NaN,21108,270354,249246,12,1181


## Analysis: by site type

In [ ]:
pivotbysitetype = last5yrs_wdeflator.pivot_table(
    index="sitetypeclean",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

pivotbysitetype

year_range,201819,201920,202021,202122,202223
sitetypeclean,,,,,
Ambulance services,NaN,22749546,24951814,NaN,NaN
Community hospital (with inpatient beds),154314166,186920727,206144929,254435345,256560258
General acute hospital,6586989290,9199038278,8777737304,8851716230,9564620691
Learning Disabilities,12647528,1766020,2691638,4676079,4265318
Mental Health (including Specialist services),210259678,273261991,385837870,438861497,450476548
Mental Health and Learning Disabilities,21517261,26486745,39186522,62446942,54751879
Mixed service hospital,131342585,191470816,163940175,213307278,219073001
Non inpatient,NaN,NaN,NaN,284435020,342851564
Other Reportable Site,352407163,431991980,476370116,139019039,199631031


In [ ]:
pivotbysitetypeHIGHRISK = last5yrs_wdeflator[
    (last5yrs_wdeflator.measureclean == 'Cost to eradicate high risk backlog (GBP)')
    ].pivot_table(
    index=["sitetypeclean","measureclean"],
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "year_range")

pivotbysitetypeHIGHRISK

,year_range,201819,201920,202021,202122,202223
sitetypeclean,measureclean,,,,,
Ambulance services,Cost to eradicate high risk backlog (GBP),NaN,1448176,485462,NaN,NaN
Community hospital (with inpatient beds),Cost to eradicate high risk backlog (GBP),8158233,42651216,41283366,66444568,25166629
General acute hospital,Cost to eradicate high risk backlog (GBP),1223230508,1611093992,1534285361,1592213579,2099169922
Learning Disabilities,Cost to eradicate high risk backlog (GBP),176336,156945,100398,935351,201000
Mental Health (including Specialist services),Cost to eradicate high risk backlog (GBP),14636905,18340843,24216749,35304025,21101866
Mental Health and Learning Disabilities,Cost to eradicate high risk backlog (GBP),485264,552029,197424,13698738,7481174
Mixed service hospital,Cost to eradicate high risk backlog (GBP),19172977,10718504,20131032,57201382,48189877
Non inpatient,Cost to eradicate high risk backlog (GBP),NaN,NaN,NaN,40218476,50419403
Other Reportable Site,Cost to eradicate high risk backlog (GBP),24382349,48355187,59504481,11518152,31604931


### Create version for chart

When creating a chart, we need the years on the index axis, so we need to switch those two parameters to get that:

In [ ]:
pivotbysitetypeHIGHRISK = last5yrs_wdeflator[
    (last5yrs_wdeflator.measureclean == 'Cost to eradicate high risk backlog (GBP)')
    ].pivot_table(
    index="year_range",
    values="inflation_adjustedOPI",
    aggfunc="sum",
    columns = "sitetypeclean")

pivotbysitetypeHIGHRISK

sitetypeclean,Ambulance services,Community hospital (with inpatient beds),General acute hospital,Learning Disabilities,Mental Health (including Specialist services),Mental Health and Learning Disabilities,Mixed service hospital,Non inpatient,Other Reportable Site,Other inpatient,Specialist hospital (acute only),Support Facility,Unoccupied
year_range,,,,,,,,,,,,,
201819,NaN,8158233,1223230508,176336,14636905,485264,19172977,NaN,24382349,207102,32213857,NaN,NaN
201920,1448176,42651216,1611093992,156945,18340843,552029,10718504,NaN,48355187,685246,68295773,NaN,NaN
202021,485462,41283366,1534285361,100398,24216749,197424,20131032,NaN,59504481,476208,125711504,NaN,NaN
202122,NaN,66444568,1592213579,935351,35304025,13698738,57201382,40218476,11518152,394619,74404463,233635,13679298
202223,NaN,25166629,2099169922,201000,21101866,7481174,48189877,50419403,31604931,1260435,84532778,7907099,6188080


In [ ]:
pivotbysitetypeHIGHRISK.to_csv("pivotbysitetypeHIGHRISK.csv")

## Export multiple sheets to an Excel spreadsheet

We can use `ExcelWriter` to create an XLSX file with different sheets containing different dataframes.

In [ ]:
# Create a Pandas Excel writer using 'openpyxl' as the engine
with pd.ExcelWriter('hospitalbuildingsANALYSIS.xlsx', engine='openpyxl') as writer:
    # Save each DataFrame to a separate sheet in the Excel workbook
    last5yrs_wdeflator.to_excel(writer, sheet_name='last5yrs', index=True)
    pivotbyregionHIGHRISK.to_excel(writer, sheet_name='pivotbyregionHIGHRISK', index=True)
    pivotbyyear_category.to_excel(writer, sheet_name='pivotbyyear_category', index=True)
    costs_hosp_only.to_excel(writer, sheet_name='costs_hosp_only', index=True)
    pivotbyHospitalHIGHRISK.to_excel(writer, sheet_name='pivotbyHospitalHIGHRISK', index=True)
    pivotbyHospitalALLRISK.to_excel(writer, sheet_name='pivotbyHospitalALLRISK', index=True)
    pivotbyTrustOLD_GACONLY_RISK.to_excel(writer, sheet_name='pivotbyTrustOLD_GACONLY_RISK', index=True)
    pivotbyTrust_GACONLY_RISK.to_excel(writer, sheet_name='pivotbyTrust_GACONLY_RISK', index=True)
    pivotbyTrust_GACONLY_RISKwHOSP.to_excel(writer, sheet_name='pivotbyTrust_GACONLY_RISKwHOSP', index=True)
    pivotbyTrustRISKDETAIL.to_excel(writer, sheet_name='pivotbyTrustRISKDETAIL', index=True)
    namenotes.to_excel(writer, sheet_name='namenotes', index=True)

#download the file
files.download('hospitalbuildingsANALYSIS.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>